In [2]:
import imaplib
import email
from datetime import datetime, timedelta, timezone
from email.header import decode_header
from email.utils import parsedate_to_datetime,getaddresses

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [9]:
# create an IMAP4_SSL class instance
imap = imaplib.IMAP4_SSL("imap.gmail.com")

# login with your Gmail account credentials
user_email = os.environ.get("IMAP_EMAIL")
password = os.environ.get("IMAP_PASSWORD")
imap.login(user_email, password)

('OK', [b'devansh.colab123@gmail.com authenticated (Success)'])

In [5]:
for i in imap.list()[1]:
    l = i.decode().split(' "/" ')
    print(l)

['(\\HasNoChildren)', '"INBOX"']
['(\\HasChildren \\Noselect)', '"[Gmail]"']
['(\\All \\HasNoChildren)', '"[Gmail]/All Mail"']
['(\\Drafts \\HasNoChildren)', '"[Gmail]/Drafts"']
['(\\HasNoChildren \\Important)', '"[Gmail]/Important"']
['(\\HasNoChildren \\Sent)', '"[Gmail]/Sent Mail"']
['(\\HasNoChildren \\Junk)', '"[Gmail]/Spam"']
['(\\Flagged \\HasNoChildren)', '"[Gmail]/Starred"']
['(\\HasNoChildren \\Trash)', '"[Gmail]/Trash"']


In [6]:
imap.select('"INBOX"')
status, messages = imap.search(None, "UNSEEN")

# iterate over the messages and retrieve their contents
for num in messages[0].split():
    print(num)
    _, msg = imap.fetch(num, "(RFC822)")
    message = email.message_from_bytes(msg[0][1])

    if message.is_multipart():
        for part in message.walk():
            if part.get_content_type() == "text/plain":
                content = part.get_payload(decode=True)
                body = content.decode("utf-8")
                break
            else:
                # if no text/plain part was found, use the first part as the body
                content = message.get_payload(0).get_payload(decode=True)
                body = content.decode("utf-8")
    else:
        # if the message is not multipart, just use the message body
        content = message.get_payload(decode=True)
        body = content.decode("utf-8")

    # print the message details
    subject = decode_header(message["Subject"])[0][0]

    print("Subject:", subject)
    print("From:", message["From"])
    print("Date:", message["Date"])
    print("Body:", body)

In [ ]:
status, msg = imap.fetch(b'22', "(RFC822 BODY[TEXT])")
message = email.message_from_bytes(msg[0][1])

print(status)
print(getaddresses(message.get_all("From", [])))
print(message["Date"])
print(parsedate_to_datetime(message["Date"]).timestamp())
print(decode_header(message["Message-ID"]))

OK
[('Devansh Rathor', 'devanshrathor123@gmail.com')]
Thu, 12 Dec 2024 17:20:00 +0530
1734004200.0
[('<CAHZGE=4rPtoKRm28xPxGG3apE3nyCH6N9rkM+Ek8whEAEKBhhA@mail.gmail.com>', None)]


In [8]:
def fetch_emails(user_email, password):
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    imap.login(user_email, password)
    imap.select('"INBOX"')
    status, messages = imap.search(None, "UNSEEN")

    emails = []

    # iterate over the messages and retrieve their contents
    for num in messages[0].split():
        _, msg = imap.fetch(num, "(RFC822)")
        message = email.message_from_bytes(msg[0][1])

        print(getaddresses(message.get_all("From", [])))
        # print the message details
        emails.append(
            {
                "timestamp": parsedate_to_datetime(message["Date"]),
                "index": int(num),
                "sender": getaddresses(message.get_all("From", [])),
                "subject": decode_header(message["Subject"])[0][0],
            }
        )
    imap.close()
    return emails

In [9]:
# login with your Gmail account credentials
user_email = os.environ.get("IMAP_EMAIL")
password = os.environ.get("IMAP_PASSWORD")
fetch_emails(user_email, password)

[]